In [9]:
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install scikit-learn
# %pip install tensorflow
# %pip install -U imbalanced-learn

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Dropout, LayerNormalization, MultiHeadAttention, Input
from tensorflow.keras.layers import Attention, Reshape
from tensorflow.keras.models import Model

In [11]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import models
import utils
import data_processing


In [12]:
COMMODITY = 'copper'

DATE_COLUMN = 'Date'
VALUE_COLUMN = 'Value'  
QUANTITY_COLUMN = 'Std. Quantity (KG)'
UNIT_RATE_COLUMN = 'Std. Unit Rate ($/KG)'
BRENT_OIL_COLUMN = 'Brent Oil Value'
WTI_OIL_COLUMN = 'WTI Oil Value'
SHIP_COUNT_COLUMN = 'ship_count'
PORT_COUNT_COLUMN = 'popular_port_count'

VALUE_SPIKES_COLUMN = 'Value Spikes'  
QUANTITY_SPIKES_COLUMN = 'Std. Quantity (KG) Spikes'
UNIT_RATE_SPIKES_COLUMN = 'Std. Unit Rate ($/KG) Spikes'
BRENT_OIL_SPIKES_COLUMN = 'Brent Oil Value Spikes'
WTI_OIL_SPIKES_COLUMN = 'WTI Oil Value Spikes'
SHIP_COUNT_SPIKES_COLUMN = 'Ship Count Spikes'
PORT_COUNT_SPIKES_COLUMN = 'Port Count Spikes'

FEATURE_COLUMNS = [VALUE_COLUMN, QUANTITY_COLUMN, UNIT_RATE_COLUMN,  WTI_OIL_COLUMN, BRENT_OIL_COLUMN, SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN]
# FEATURE_COLUMNS = [VALUE_COLUMN, QUANTITY_COLUMN, UNIT_RATE_COLUMN,  WTI_OIL_SPIKES_COLUMN, BRENT_OIL_SPIKES_COLUMN]
# FEATURE_COLUMNS = [VALUE_COLUMN, QUANTITY_COLUMN, UNIT_RATE_COLUMN,  WTI_OIL_COLUMN, BRENT_OIL_COLUMN, SHIP_COUNT_SPIKES_COLUMN, PORT_COUNT_SPIKES_COLUMN]
TARGET_COLUMN = 'Price'

ORIGIN_COUNTRY_COLUMN = 'Country of Origin'
DEST_COUNTRY_COLUMN = 'Country of Destination'

PETROL_FILE_PATH = '../../volza/petroleum/petrol_crude_oil_spot_price.csv'
VOLZA_FILE_PATH = f'../../volza/{COMMODITY}/{COMMODITY}.csv'
PRICE_FILE_PATH = f"../../volza/{COMMODITY}/{COMMODITY}_prices.csv"
AIS_POPULAR_FILE_PATH = f'../../ais/ais_ml_features.csv' 

OUTPUT_FOLDER_PATH = os.path.join(COMMODITY, "oil")

SPIKES_THRESHOLD = 2
SPIKES_WINDOW_SIZE = 20
BIN_COUNT = 10
FILL_METHOD = 'ffill'

RANDOM_STATE = 42

if not os.path.exists(OUTPUT_FOLDER_PATH):
    os.mkdir(OUTPUT_FOLDER_PATH)

## Dataframe Prep

In [13]:
from datetime import datetime

#Formatting the date and price for Volza data
volza_pd = pd.read_csv(VOLZA_FILE_PATH)
volza_pd = volza_pd[(volza_pd["Country of Origin"].notnull()) & (volza_pd["Country of Destination"].notnull())]
volza_pd = volza_pd.rename(columns={'Unnamed: 0': 'ID'})
volza_pd['Date'] = volza_pd['Date'].apply(lambda x: x.split(' ')[0])
volza_pd['Date'] = pd.to_datetime(volza_pd['Date'], errors='raise', format='%Y-%m-%d')
volza_pd = utils.convert_to_kg(volza_pd)
volza_pd.head(3)

,0,Date,HS Code,Product Description,Consignee,Notify Party Name,Shipper,Std. Quantity,Std. Unit,Standard Unit Rate INR,...,Freight Term,Marks Number,HS Product Description,Gross Weight,Consignee Address,Shipper Address,Notify Party Address,Country Name,Std. Quantity (KG),Std. Unit Rate ($/KG)
0,1,2019-04-01,74020000,"COPPER, UNREFINED; COPPER ANODES FOR ELECTROLY...",NaN,NaN,NaN,0.007,TON,-,...,-,-,NaN,0.0,NaN,NaN,NaN,Lithuania T1 Import,6.350295,12.208882
1,2,2019-04-01,74020000,"COPPER, UNREFINED; COPPER ANODES FOR ELECTROLY...",NaN,NaN,NaN,0.021,TON,-,...,-,-,NaN,0.0,NaN,NaN,NaN,Portugal T1 Import,19.050885,144.438959
2,3,2019-04-01,74020000,"COPPER, UNREFINED; COPPER ANODES FOR ELECTROLY...",NaN,NaN,NaN,0.001,TON,-,...,-,-,NaN,0.0,NaN,NaN,NaN,France T1 Import,0.907185,22.288728


In [14]:
#Preprocessing the AIS data
ais_popular_pd = pd.read_csv(AIS_POPULAR_FILE_PATH)
ais_popular_pd['Date'] = pd.to_datetime(ais_popular_pd['Date'])
ais_popular_pd.head(3)


,Date,ship_count,popular_port,popular_port_count
0,2020-11-10,8,LTKLJ,18
1,2020-11-12,20,IDSKP,8
2,2020-11-29,9,CNSHA,2


In [15]:
#Preprocessing the price data
prices_pd = pd.read_csv(PRICE_FILE_PATH)
prices_pd['Date'] = prices_pd['Date'].apply(lambda x: datetime.strptime(x, "%b %d, %Y").strftime("%Y-%m-%d"))
prices_pd['Date'] = pd.to_datetime(prices_pd['Date'], errors='raise', format='%Y-%m-%d')
prices_pd['Price'] = prices_pd['Price'].astype(float)
prices_pd = prices_pd[['Date','Price']]
prices_pd.head(3)

,Date,Price
0,2022-12-30,3.8055
1,2022-12-29,3.8210
2,2022-12-28,3.8320


In [16]:
#Aggregate volza data by day
date_wise_volza = volza_pd.groupby("Date")[[VALUE_COLUMN,QUANTITY_COLUMN,'Gross Weight']].sum()

In [17]:
# Avg of Commodity Price in Volza
avg_price_volza = volza_pd.groupby('Date')[UNIT_RATE_COLUMN].mean()
date_wise_volza = date_wise_volza.join(avg_price_volza, how='left')
date_wise_volza

,Value,Std. Quantity (KG),Gross Weight,Std. Unit Rate ($/KG)
Date,,,,
2019-01-01,3.580679e+08,5.057622e+07,512426.9,51.222824
2019-01-02,2.899774e+07,4.504496e+06,4507752.0,6.420416
2019-01-03,1.652543e+07,2.427492e+06,2428782.0,9.079466
2019-01-04,1.381037e+05,1.978600e+04,0.0,5.651829
2019-01-06,1.854971e+07,3.025544e+06,0.0,3947.152515
...,...,...,...,...
2022-12-28,3.660210e+07,4.590079e+06,2301364.0,7.600228
2022-12-29,9.597299e+06,2.434564e+06,2415.6,13.516123
2022-12-30,1.148950e+08,2.272949e+06,360.0,371.032957


In [18]:
# Petroleum data prep
petrol_df = pd.read_csv(PETROL_FILE_PATH, delimiter=';', on_bad_lines='warn')
petrol_df['Date'] = pd.to_datetime(petrol_df['Date'])

# Split based on types of oil
brent_df = petrol_df[petrol_df['product-name']=='UK Brent Crude Oil']
wti_df = petrol_df[petrol_df['product-name']=='WTI Crude Oil']

brent_df.rename(columns={'Value':'Brent Oil Value'}, inplace=True)
wti_df.rename(columns={'Value':'WTI Oil Value'}, inplace=True)


/tmp/ipykernel_12624/273638861.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brent_df.rename(columns={'Value':'Brent Oil Value'}, inplace=True)
/tmp/ipykernel_12624/273638861.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wti_df.rename(columns={'Value':'WTI Oil Value'}, inplace=True)


In [19]:
# Combining dataframes
prices_pd = prices_pd.set_index('Date')
ais_popular_pd = ais_popular_pd.set_index('Date')
date_wise_volza = date_wise_volza.join(ais_popular_pd, how="left").fillna(method=FILL_METHOD)
aggregated_df = date_wise_volza.join(prices_pd, how="left").fillna(method=FILL_METHOD)
aggregated_df = aggregated_df.merge(brent_df[[DATE_COLUMN, BRENT_OIL_COLUMN]], on='Date', how='left').fillna(method=FILL_METHOD)
aggregated_df = aggregated_df.merge(wti_df[[DATE_COLUMN, WTI_OIL_COLUMN]], on='Date', how='left').fillna(method=FILL_METHOD)
aggregated_df.head()

,Date,Value,Std. Quantity (KG),Gross Weight,Std. Unit Rate ($/KG),ship_count,popular_port,popular_port_count,Price,Brent Oil Value,WTI Oil Value
0,2019-01-01,3.580679e+08,5.057622e+07,512426.9,51.222824,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,2.899774e+07,4.504496e+06,4507752.0,6.420416,NaN,NaN,NaN,2.6250,54.06,46.31
2,2019-01-03,1.652543e+07,2.427492e+06,2428782.0,9.079466,NaN,NaN,NaN,2.5705,53.23,46.92
3,2019-01-04,1.381037e+05,1.978600e+04,0.0,5.651829,NaN,NaN,NaN,2.6515,55.64,47.76
4,2019-01-06,1.854971e+07,3.025544e+06,0.0,3947.152515,NaN,NaN,NaN,2.6515,55.64,47.76


In [20]:
def detect_spikes(df, column, window_size):
    ## Detecting spikes
    moving_avg = df[column].rolling(window=window_size).mean()
    std_dev = df[column].rolling(window=window_size).std()

    # Set a threshold to identify spikes
    return (abs(aggregated_df[column] - moving_avg) > SPIKES_THRESHOLD * std_dev).astype(int)

# aggregated_df['spikes'] = detect_spikes(aggregated_df, 'Price')
# print("SPIKES : NON SPIKES = ")
# print(aggregated_df['spikes'].value_counts())
# print("PERCENT OF SPIKES", aggregated_df['spikes'].value_counts()[1]/len(aggregated_df))

# **Detect spikes**

In [21]:
# aggregated_df[VALUE_SPIKES_COLUMN] = detect_spikes(aggregated_df, VALUE_COLUMN)
# aggregated_df[QUANTITY_SPIKES_COLUMN] = detect_spikes(aggregated_df, QUANTITY_COLUMN)
# aggregated_df[UNIT_RATE_SPIKES_COLUMN] = detect_spikes(aggregated_df, UNIT_RATE_COLUMN)
# aggregated_df[WTI_OIL_SPIKES_COLUMN] = detect_spikes(aggregated_df, WTI_OIL_COLUMN)
# aggregated_df[BRENT_OIL_SPIKES_COLUMN] = detect_spikes(aggregated_df, BRENT_OIL_COLUMN)
# aggregated_df[SHIP_COUNT_SPIKES_COLUMN] = detect_spikes(aggregated_df, SHIP_COUNT_COLUMN)
# aggregated_df[PORT_COUNT_SPIKES_COLUMN] = detect_spikes(aggregated_df, PORT_COUNT_COLUMN)

# #Visualise Dataset

# # Plotting the graph
# fig, ax1 = plt.subplots(figsize=(12, 6))

# # Plotting 'Value', 'Quantity', and 'Gross Weight' on the left y-axis
# ax1.plot(aggregated_df.index, aggregated_df[VALUE_SPIKES_COLUMN], label='Value Spikes', color='b')
# ax1.plot(aggregated_df.index, aggregated_df[QUANTITY_SPIKES_COLUMN], label='Quantity Spikes', color='g')
# ax1.plot(aggregated_df.index, aggregated_df[UNIT_RATE_SPIKES_COLUMN], label='Unit Rate Spikes', color='k')
# ax1.plot(aggregated_df.index, aggregated_df[BRENT_OIL_SPIKES_COLUMN], label='Brent Oil Value Spikes', color='m')
# ax1.plot(aggregated_df.index, aggregated_df[WTI_OIL_SPIKES_COLUMN], label='WTI Oil Value Spikes', color='c')
# ax1.plot(aggregated_df.index, aggregated_df[SHIP_COUNT_COLUMN], label='Ship Count Spikes', color='darkorange')
# ax1.plot(aggregated_df.index, aggregated_df[PORT_COUNT_COLUMN], label='Port Count Value Spikes', color='violet')

# ax1.set_xlabel('Date')
# ax1.set_ylabel('Value / Quantity / Gross Weight / Brent Oil Value / WTI Oil Value / Ship Count / Port Count ', color='b')
# ax1.tick_params('y', colors='b')

# # Creating a second y-axis for 'Price'
# ax2 = ax1.twinx()
# ax2.plot(aggregated_df.index, aggregated_df['Price'], label='Price', color='orange')
# ax2.set_ylabel('Price', color='orange')
# ax2.tick_params('y', colors='orange')

# # Display legend
# fig.tight_layout()
# fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))

# # Display the graph
# # plt.show()

In [22]:
#remove date 2020-01-01
aggregated_df = aggregated_df[aggregated_df.index != '2020-01-01']
aggregated_df

,Date,Value,Std. Quantity (KG),Gross Weight,Std. Unit Rate ($/KG),ship_count,popular_port,popular_port_count,Price,Brent Oil Value,WTI Oil Value
0,2019-01-01,3.580679e+08,5.057622e+07,512426.9,51.222824,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,2.899774e+07,4.504496e+06,4507752.0,6.420416,NaN,NaN,NaN,2.6250,54.06,46.31
2,2019-01-03,1.652543e+07,2.427492e+06,2428782.0,9.079466,NaN,NaN,NaN,2.5705,53.23,46.92
3,2019-01-04,1.381037e+05,1.978600e+04,0.0,5.651829,NaN,NaN,NaN,2.6515,55.64,47.76
4,2019-01-06,1.854971e+07,3.025544e+06,0.0,3947.152515,NaN,NaN,NaN,2.6515,55.64,47.76
...,...,...,...,...,...,...,...,...,...,...,...
1233,2022-12-28,3.660210e+07,4.590079e+06,2301364.0,7.600228,26021.0,SGSIN,9233.0,3.8320,81.70,78.89
1234,2022-12-29,9.597299e+06,2.434564e+06,2415.6,13.516123,24039.0,SGSIN,8861.0,3.8210,80.96,78.43
1235,2022-12-30,1.148950e+08,2.272949e+06,360.0,371.032957,19952.0,SGSIN,8118.0,3.8055,82.82,80.16
1236,2022-12-31,1.290664e+07,1.515365e+06,0.0,8.285327,8925.0,USMSY,2673.0,3.8055,82.82,80.16


In [23]:
# #Visualise Dataset
# # Plotting the graph
# fig, ax1 = plt.subplots(figsize=(12, 6))

# # Plotting 'Value', 'Quantity', and 'Gross Weight' on the left y-axis
# ax1.plot(aggregated_df.index, aggregated_df[VALUE_COLUMN], label='Value', color='b')
# ax1.plot(aggregated_df.index, aggregated_df[QUANTITY_COLUMN], label='Quantity', color='g')
# ax1.plot(aggregated_df.index, aggregated_df[UNIT_RATE_COLUMN], label='Unit Rate', color='k')
# ax1.plot(aggregated_df.index, aggregated_df[BRENT_OIL_COLUMN], label='Brent Oil Value', color='m')
# ax1.plot(aggregated_df.index, aggregated_df[WTI_OIL_COLUMN], label='WTI Oil Value', color='c')
# ax1.plot(aggregated_df.index, aggregated_df[SHIP_COUNT_COLUMN], label='Ship Count Value', color='darkorange')
# ax1.plot(aggregated_df.index, aggregated_df[PORT_COUNT_COLUMN], label='Port Count Value', color='violet')

# ax1.set_xlabel('Date')
# ax1.set_ylabel('Value / Quantity / Gross Weight', color='b')
# ax1.tick_params('y', colors='b')

# # Creating a second y-axis for 'Price'
# ax2 = ax1.twinx()
# ax2.plot(aggregated_df.index, aggregated_df['Price'], label='Price', color='orange')
# ax2.set_ylabel('Price', color='orange')
# ax2.tick_params('y', colors='orange')

# # Display legend
# fig.tight_layout()
# fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))

# # Display the graph
# # plt.show()

In [24]:
# # Plotting the price data
# plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
# plt.plot(aggregated_df.index, aggregated_df['Price'], label='Price', color='blue')

# # Highlighting spikes
# spike_indices = aggregated_df[aggregated_df['spikes'] == 1].index
# spike_prices = aggregated_df.loc[spike_indices, 'Price']
# plt.scatter(spike_indices, spike_prices, color='red', marker='^', label='Spikes')

# # Adding labels and title
# plt.xlabel('Date')
# plt.ylabel('Price')
# plt.title('Price Data with Spikes')
# plt.legend()

# # Display the plot
# # plt.show()

## Baseline

In [25]:
# # Count % of spikes 
# total_spikes = aggregated_df['spikes'].sum()
# total_data_points = len(aggregated_df)
# percentage_of_spikes = (total_spikes / total_data_points) * 100

# print(f"Percentage of Spikes: {percentage_of_spikes:.2f}%")

In [26]:
# from sklearn.metrics import precision_score, recall_score

# # Probability of spike
# spike_prob = aggregated_df['spikes'].mean()

# # Random baseline predictions
# random_predictions = np.random.choice([0, 1], size=len(aggregated_df), p=[1-spike_prob, spike_prob])

# # Calculate precision and recall for the random baseline
# random_precision = precision_score(aggregated_df['spikes'], random_predictions)
# random_recall = recall_score(aggregated_df['spikes'], random_predictions)

# print(f"Random Guessing Precision: {random_precision}")
# print(f"Random Guessing Recall: {random_recall}")


## Data Prep for Classification

In [27]:
COLUMNS = FEATURE_COLUMNS + [TARGET_COLUMN]
print(COLUMNS)

['Value', 'Std. Quantity (KG)', 'Std. Unit Rate ($/KG)', 'WTI Oil Value', 'Brent Oil Value', 'ship_count', 'popular_port_count', 'Price']


In [28]:
# # Discretize
# from sklearn.preprocessing import KBinsDiscretizer

# def discretize(df, columns, bins):
#     est = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy='kmeans')
#     df[columns] = est.fit_transform(df[columns])
#     return df

# # FEATURES_1 = [VALUE_COLUMN, QUANTITY_COLUMN, UNIT_RATE_COLUMN]
# # FEATURES_2 = [WTI_OIL_COLUMN, BRENT_OIL_COLUMN]

# test_df = aggregated_df.copy()
# test_df[FEATURE_COLUMNS] = test_df[FEATURE_COLUMNS].fillna(0)

# # test_df = discretize(test_df, FEATURES_1, 2)
# # test_df = discretize(test_df, FEATURES_2, BIN_COUNT)
# discretized_df = discretize(test_df, FEATURE_COLUMNS, BIN_COUNT)
# # discretized_df = test_df.copy()
# test_df.head(2)


In [29]:
# fig, ax1 = plt.subplots(figsize=(12, 6))

# # Plotting 'Value', 'Quantity', and 'Gross Weight' on the left y-axis
# ax1.plot(test_df.index, test_df[VALUE_COLUMN], label='Value', color='b')
# ax1.plot(test_df.index, test_df[QUANTITY_COLUMN], label='Quantity', color='g')
# ax1.plot(test_df.index, test_df[UNIT_RATE_COLUMN], label='Unit Rate', color='k')
# ax1.plot(test_df.index, test_df[BRENT_OIL_COLUMN], label='Brent Oil Value', color='m')
# ax1.plot(test_df.index, test_df[WTI_OIL_COLUMN], label='WTI Oil Value', color='c')
# ax1.plot(test_df.index, test_df[SHIP_COUNT_COLUMN], label='Ship Count Value', color='darkorange')
# ax1.plot(test_df.index, test_df[PORT_COUNT_COLUMN], label='Port Count Value', color='violet')

# ax1.set_xlabel('Date')
# ax1.set_ylabel('Value / Quantity / Gross Weight', color='b')
# ax1.tick_params('y', colors='b')

# # Creating a second y-axis for 'Price'
# ax2 = ax1.twinx()
# ax2.plot(test_df.index, test_df['Price'], label='Price', color='orange')
# ax2.set_ylabel('Price', color='orange')
# ax2.tick_params('y', colors='orange')

# # Display legend
# fig.tight_layout()
# fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))

# # Display the graph
# plt.show()

In [30]:
# # Convert the discretized data into a DataFrame
# discretized_df = pd.DataFrame(discretized_df, columns=FEATURE_COLUMNS)

# unique_values = discretized_df[VALUE_COLUMN].fillna(method=FILL_METHOD).unique()
# print(unique_values)

# bin_counts = {col: discretized_df[col].value_counts() for col in FEATURE_COLUMNS}

# # Plotting
# plt.figure(figsize=(15, len(FEATURE_COLUMNS) * 5))

# for i, column in enumerate(FEATURE_COLUMNS):
#     plt.subplot(len(FEATURE_COLUMNS), 1, i + 1)
#     bin_counts[column].sort_index().plot(kind='bar', ax=plt.gca())

#     plt.title(f'{column} Distribution')
#     plt.ylabel('Frequency')
#     plt.xlabel('Bins')

# plt.tight_layout()
# plt.show()


In [31]:
# Clean up before passing to Arima
initial_row_count = aggregated_df.shape[0]

columns_of_interest = ['Brent Oil Value', 'WTI Oil Value', 'Price']  # Add other columns as necessary

aggregated_df = aggregated_df.dropna(subset=columns_of_interest)

rows_dropped = initial_row_count - aggregated_df.shape[0]

print(f"Rows dropped due to NaN values: {rows_dropped}")

Rows dropped due to NaN values: 1


## Train / Test Set Up

In [32]:
from sklearn.model_selection import train_test_split
import pmdarima as pm

# Fit an Auto ARIMA model to the 'Price' series
model = pm.auto_arima(aggregated_df['Price'], seasonal=True, m=12, suppress_warnings=True, stepwise=True, error_action='ignore')

# Forecast the series using the model (in-sample prediction)
forecast = model.predict_in_sample()

# Calculate residuals (difference between actual and forecasted values)
residuals = aggregated_df['Price'] - forecast

# Append residuals to DataFrame as a new feature (using residuals as a way to detect spike / anomaly)
aggregated_df = aggregated_df.copy()
aggregated_df['ARIMA_Residuals'] = residuals

In [33]:
# Trying out different window sizes
SPIKE_WINDOW_SIZES = [10, 20, 30, 40, 60, 80, 100]
results_dfs = []

for window_size in SPIKE_WINDOW_SIZES:
    print(f"Evaluating window size: {window_size}")

    aggregated_df['spikes'] = detect_spikes(aggregated_df, 'Price', window_size)

    # Prepare features and target
    FEATURE_COLUMNS = [TARGET_COLUMN, 'ARIMA_Residuals', BRENT_OIL_COLUMN, WTI_OIL_COLUMN]  # Adjust as needed
    X, y = data_processing.prepare_features_and_target(aggregated_df, FEATURE_COLUMNS, 'spikes')

    # Split data 
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, shuffle=False)

    # Scale features
    X_train_scaled, X_test_scaled = data_processing.scale_features(X_train_raw, X_test_raw)

    # Create sequences
    X_train, y_train = data_processing.create_sequences(X_train_scaled, y_train, window_size)
    X_test, y_test = data_processing.create_sequences(X_test_scaled, y_test, window_size)

    print(f'X train shape: {X_train.shape}')

    output_file_path = os.path.join(OUTPUT_FOLDER_PATH, f'results_{window_size}.csv')
    results_df = models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)
    results_df['Window Size'] = window_size
    results_dfs.append(results_df)

Evaluating window size: 10
X train shape: (980, 10, 4)


2024-02-21 22:36:02.847259: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2746 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5
2024-02-21 22:36:05.089357: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-02-21 22:36:05.192208: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f38ba6b6ad0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-21 22:36:05.192243: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-02-21 22:36:05.197324: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-21 22:36:05.372500: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled

8/8 [==============================] - 0s 2ms/step
Evaluating window size: 20
X train shape: (970, 20, 4)
8/8 [==============================] - 0s 1ms/step
Evaluating window size: 30
X train shape: (960, 30, 4)
7/7 [==============================] - 0s 1ms/step
Evaluating window size: 40
X train shape: (950, 40, 4)
7/7 [==============================] - 0s 1ms/step
Evaluating window size: 60
X train shape: (930, 60, 4)
6/6 [==============================] - 0s 2ms/step
Evaluating window size: 80
X train shape: (910, 80, 4)
6/6 [==============================] - 0s 7ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


6/6 [==============================] - 0s 9ms/step


...

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 [==============================] - 0s 5ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 [==============================] - 0s 1ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Evaluation

In [34]:
# Display all results df
for idx, df in enumerate(results_dfs):
    print(f"Results for window size: {SPIKE_WINDOW_SIZES[idx]}")
    display(df) 

Results for window size: 10


,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Window Size
0,LSTM,250 layers,0.953975,0.969298,0.982222,0.975717,0.636364,0.500000,0.560000,10
1,LSTM,200 layers,0.970711,0.978070,0.991111,0.984547,0.818182,0.642857,0.720000,10
2,LSTM,100 layers,0.970711,0.978070,0.991111,0.984547,0.818182,0.642857,0.720000,10
3,LSTM,50 layers,0.966527,0.982222,0.982222,0.982222,0.714286,0.714286,0.714286,10
4,CNN with Attention,"32 filters, kernel size 7",0.966527,0.977974,0.986667,0.982301,0.750000,0.642857,0.692308,10
5,CNN with Attention,"32 filters, kernel size 5",0.958159,0.977778,0.977778,0.977778,0.642857,0.642857,0.642857,10
6,CNN with Attention,"32 filters, kernel size 3",0.962343,0.965517,0.995556,0.980306,0.857143,0.428571,0.571429,10
7,CNN with Attention,"64 filters, kernel size 7",0.966527,0.977974,0.986667,0.982301,0.750000,0.642857,0.692308,10
8,CNN with Attention,"64 filters, kernel size 5",0.966527,0.973799,0.991111,0.982379,0.800000,0.571429,0.666667,10
9,CNN with Attention,"64 filters, kernel size 3",0.966527,0.969697,0.995556,0.982456,0.875000,0.500000,0.636364,10


Results for window size: 20


,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Window Size
0,LSTM,250 layers,0.930131,0.961353,0.961353,0.961353,0.636364,0.636364,0.636364,20
1,LSTM,200 layers,0.877729,0.958974,0.903382,0.930348,0.411765,0.636364,0.500000,20
2,LSTM,100 layers,0.908297,0.947115,0.951691,0.949398,0.523810,0.500000,0.511628,20
3,LSTM,50 layers,0.890830,0.955000,0.922705,0.938575,0.448276,0.590909,0.509804,20
4,CNN with Attention,"32 filters, kernel size 7",0.951965,0.962264,0.985507,0.973747,0.823529,0.636364,0.717949,20
5,CNN with Attention,"32 filters, kernel size 5",0.938865,0.953052,0.980676,0.966667,0.750000,0.545455,0.631579,20
6,CNN with Attention,"32 filters, kernel size 3",0.943231,0.949074,0.990338,0.969267,0.846154,0.500000,0.628571,20
7,CNN with Attention,"64 filters, kernel size 7",0.930131,0.948357,0.975845,0.961905,0.687500,0.500000,0.578947,20
8,CNN with Attention,"64 filters, kernel size 5",0.930131,0.944186,0.980676,0.962085,0.714286,0.454545,0.555556,20
9,CNN with Attention,"64 filters, kernel size 3",0.943231,0.949074,0.990338,0.969267,0.846154,0.500000,0.628571,20


Results for window size: 30


...

,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Window Size
0,LSTM,250 layers,0.873016,0.886364,0.97500,0.928571,0.692308,0.310345,0.428571,60
1,LSTM,200 layers,0.862434,0.940789,0.89375,0.916667,0.540541,0.689655,0.606061,60
2,LSTM,100 layers,0.857143,0.923567,0.90625,0.914826,0.531250,0.586207,0.557377,60
3,LSTM,50 layers,0.724868,0.875000,0.78750,0.828947,0.244444,0.379310,0.297297,60
4,CNN with Attention,"32 filters, kernel size 7",0.793651,0.875776,0.88125,0.878505,0.321429,0.310345,0.315789,60
5,CNN with Attention,"32 filters, kernel size 5",0.835979,0.860335,0.96250,0.908555,0.400000,0.137931,0.205128,60
6,CNN with Attention,"32 filters, kernel size 3",0.693122,0.904762,0.71250,0.797203,0.269841,0.586207,0.369565,60
7,CNN with Attention,"64 filters, kernel size 7",0.740741,0.872483,0.81250,0.841424,0.250000,0.344828,0.289855,60
8,CNN with Attention,"64 filters, kernel size 5",0.465608,0.847059,0.45000,0.587755,0.153846,0.551724,0.240602,60
9,CNN with Attention,"64 filters, kernel size 3",0.746032,0.959016,0.73125,0.829787,0.358209,0.827586,0.500000,60


Results for window size: 80


,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Window Size
0,LSTM,250 layers,0.946746,0.981132,0.962963,0.971963,0.400000,0.571429,0.470588,80
1,LSTM,200 layers,0.946746,0.981132,0.962963,0.971963,0.400000,0.571429,0.470588,80
2,LSTM,100 layers,0.958580,0.975460,0.981481,0.978462,0.500000,0.428571,0.461538,80
3,LSTM,50 layers,0.976331,0.975904,1.000000,0.987805,1.000000,0.428571,0.600000,80
4,CNN with Attention,"32 filters, kernel size 7",0.964497,0.964286,1.000000,0.981818,1.000000,0.142857,0.250000,80
5,CNN with Attention,"32 filters, kernel size 5",0.958580,0.958580,1.000000,0.978852,0.000000,0.000000,0.000000,80
6,CNN with Attention,"32 filters, kernel size 3",0.958580,0.958580,1.000000,0.978852,0.000000,0.000000,0.000000,80
7,CNN with Attention,"64 filters, kernel size 7",0.958580,0.958580,1.000000,0.978852,0.000000,0.000000,0.000000,80
8,CNN with Attention,"64 filters, kernel size 5",0.970414,0.970060,1.000000,0.984802,1.000000,0.285714,0.444444,80
9,CNN with Attention,"64 filters, kernel size 3",0.958580,0.958580,1.000000,0.978852,0.000000,0.000000,0.000000,80


Results for window size: 100


,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Window Size
0,LSTM,250 layers,0.946309,0.964789,0.978571,0.971631,0.571429,0.444444,0.500000,100
1,LSTM,200 layers,0.939597,0.964539,0.971429,0.967972,0.500000,0.444444,0.470588,100
2,LSTM,100 layers,0.926174,0.957447,0.964286,0.960854,0.375000,0.333333,0.352941,100
3,LSTM,50 layers,0.872483,0.991870,0.871429,0.927757,0.307692,0.888889,0.457143,100
4,CNN with Attention,"32 filters, kernel size 7",0.939597,0.939597,1.000000,0.968858,0.000000,0.000000,0.000000,100
5,CNN with Attention,"32 filters, kernel size 5",0.939597,0.939597,1.000000,0.968858,0.000000,0.000000,0.000000,100
6,CNN with Attention,"32 filters, kernel size 3",0.939597,0.939597,1.000000,0.968858,0.000000,0.000000,0.000000,100
7,CNN with Attention,"64 filters, kernel size 7",0.939597,0.939597,1.000000,0.968858,0.000000,0.000000,0.000000,100
8,CNN with Attention,"64 filters, kernel size 5",0.939597,0.939597,1.000000,0.968858,0.000000,0.000000,0.000000,100
9,CNN with Attention,"64 filters, kernel size 3",0.939597,0.939597,1.000000,0.968858,0.000000,0.000000,0.000000,100


### **Random Under Sampling**

In [35]:
# from imblearn.under_sampling import RandomUnderSampler
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler


# time_series_df = aggregated_df.copy()

# # Drop rows with NaN in the 'spikes' column
# time_series_df = time_series_df.dropna(subset=['spikes'])
# discretized_df = discretize(time_series_df[FEATURE_COLUMNS], FEATURE_COLUMNS, BIN_COUNT)
# time_series_df[FEATURE_COLUMNS] = discretized_df

# # Extract features and target variable BEFORE creating sequences
# X = time_series_df[FEATURE_COLUMNS].values
# y = time_series_df['spikes'].values

# # Feature scaling using StandardScaler
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Apply RandomOverSampler BEFORE creating sequences
# random_under_sampler = RandomUnderSampler(random_state=RANDOM_STATE)
# X_scaled_resampled, y_resampled = random_under_sampler.fit_resample(X_scaled, y)

# # Recreate sequences with resampled data
# X_sequences_resampled, y_sequences_resampled = [], []
# for i in range(len(X_scaled_resampled) - SPIKES_WINDOW_SIZE + 1):
#     X_sequences_resampled.append(X_scaled_resampled[i:i + SPIKES_WINDOW_SIZE, :])
#     y_sequences_resampled.append(y_resampled[i + SPIKES_WINDOW_SIZE - 1])

# X_sequences_resampled, y_sequences_resampled = np.array(X_sequences_resampled), np.array(y_sequences_resampled)

# # Split the resampled sequences into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_sequences_resampled, y_sequences_resampled, test_size=0.2, random_state=50)

# # Proceed with model training and evaluation
# evaluate_all_models(RUS_OUTPUT_PATH)


### **Random Over Sampling**

In [36]:
# from imblearn.over_sampling import RandomOverSampler

# time_series_df = aggregated_df.copy()

# # Drop rows with NaN in the 'spikes' column
# time_series_df = time_series_df.dropna(subset=['spikes'])
# discretized_df = discretize(time_series_df[FEATURE_COLUMNS], FEATURE_COLUMNS, BIN_COUNT)
# time_series_df[FEATURE_COLUMNS] = discretized_df

# # Extract features and target variable BEFORE creating sequences
# X = time_series_df[FEATURE_COLUMNS].values
# y = time_series_df['spikes'].values

# # Feature scaling using StandardScaler
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Apply RandomUnderSampler BEFORE creating sequences
# random_over_sampler = RandomOverSampler(random_state=RANDOM_STATE)
# X_scaled_resampled, y_resampled = random_over_sampler.fit_resample(X_scaled, y)

# # Recreate sequences with resampled data
# X_sequences_resampled, y_sequences_resampled = [], []
# for i in range(len(X_scaled_resampled) - SPIKES_WINDOW_SIZE + 1):
#     X_sequences_resampled.append(X_scaled_resampled[i:i + SPIKES_WINDOW_SIZE, :])
#     y_sequences_resampled.append(y_resampled[i + SPIKES_WINDOW_SIZE - 1])

# X_sequences_resampled, y_sequences_resampled = np.array(X_sequences_resampled), np.array(y_sequences_resampled)

# # Split the resampled sequences into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_sequences_resampled, y_sequences_resampled, test_size=0.2, random_state=50)

# evaluate_all_models(ROS_OUTPUT_PATH)